In [4]:
import os
import pandas as pd
import sqlalchemy
import sklearn
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

connection_string = os.getenv("LINODE_CONN_STRING")
connection_string = os.getenv("LINODE_CONN_STRING")
conn = sqlalchemy.create_engine(os.getenv("LINODE_CONN_STRING")).connect()

In [5]:
all_games = pd.read_sql("select * from sports_data.game_stats_v2", conn)

In [6]:
# so we have all the games from each season. What features can we engineer?
# 1. regular season win/loss ratio
# 2. conference strength (win/loss ratio for the whole conference)
# 3. last ten game win/loss ratio

# that's pretty good for now

In [7]:
# regular season win/loss ratio
season_avgs = all_games.groupby(['team', 'season','Win_Loss'], as_index=False).size()

In [8]:
season_records = season_avgs.pivot_table(index=['team', 'season'], columns='Win_Loss', values='size', fill_value=0).reset_index()[['team', 'season', 'L', 'W']]
season_records.rename(columns={'L': 'losses', 'W': 'wins'}, inplace=True)

In [9]:
tourney_data = pd.read_sql("select * from sports_data.tourney_games_with_season_stats", conn)

In [10]:
season_records['win_loss_ratio'] = season_records['wins'] / (season_records['wins'] + season_records['losses'])

In [11]:
interconference_games = all_games[all_games['conference'] != all_games['opponent_conference']]

In [12]:
conference_averages = interconference_games.groupby(['conference', 'season','Win_Loss'], as_index=False).size()

In [13]:
conference_records = conference_averages.pivot_table(index=['conference', 'season'], columns='Win_Loss', values='size', fill_value=0).reset_index()[['conference', 'season', 'L', 'W']]

In [14]:
conference_records.rename(columns={'L': 'losses', 'W': 'wins'}, inplace=True)
conference_records['win_loss_ratio'] = conference_records['wins'] / (conference_records['wins'] + conference_records['losses'])
conference_records

Win_Loss,conference,season,losses,wins,win_loss_ratio
0,A10,2015,70.0,105.0,0.600000
1,A10,2016,64.0,117.0,0.646409
2,A10,2017,72.0,108.0,0.600000
3,A10,2018,85.0,92.0,0.519774
4,A10,2019,88.0,94.0,0.516484
...,...,...,...,...,...
317,WCC,2022,57.0,87.0,0.604167
318,WCC,2023,54.0,85.0,0.611511
319,WCC,2024,58.0,62.0,0.516667
320,ind,2023,41.0,7.0,0.145833


In [15]:
tourney_data = pd.merge(tourney_data, season_records, left_on=['team', 'season'], right_on=['team', 'season'], how='left')

In [16]:
tourney_data = pd.merge(tourney_data, season_records, left_on=['opponent', 'season'], right_on=['team', 'season'], how='left', suffixes=('', '_opponent'))

In [17]:
tourney_data

,date,team,opponent,season,location,Win_Loss,score,opponent_score,adj_offense_team,adj_defense_team,...,opponent_turnover_percent_opponent,opponent_offensive_rebound_percent_opponent,opponent_free_throw_rate_opponent,losses,wins,win_loss_ratio,team_opponent,losses_opponent,wins_opponent,win_loss_ratio_opponent
0,3/17/23,Arizona St.,TCU,2023,N,L,70,72,105.770588,93.755882,...,21.963636,30.745455,30.539394,13.0,23.0,0.638889,TCU,13.0,22.0,0.628571
1,3/17/23,TCU,Arizona St.,2023,N,W,72,70,110.433333,92.660606,...,20.541176,31.555882,34.017647,13.0,22.0,0.628571,Arizona St.,13.0,23.0,0.638889
2,3/23/23,Arkansas,Connecticut,2023,N,L,65,88,110.545455,91.821212,...,19.206061,26.109091,38.945455,14.0,22.0,0.611111,Connecticut,8.0,31.0,0.794872
3,3/23/23,Connecticut,Arkansas,2023,N,W,88,65,118.684848,92.387879,...,20.284848,28.193939,39.651515,8.0,31.0,0.794872,Arkansas,14.0,22.0,0.611111
4,3/18/23,Arkansas,Kansas,2023,N,W,72,71,110.545455,91.821212,...,20.200000,28.105882,31.544118,14.0,22.0,0.611111,Kansas,8.0,28.0,0.777778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237,3/19/15,Wofford,Arkansas,2015,N,L,53,56,102.521875,97.062500,...,22.458824,34.135294,39.347059,7.0,26.0,0.787879,Arkansas,9.0,27.0,0.750000
1238,3/20/15,Northern Iowa,Wyoming,2015,N,W,71,54,114.212121,92.445455,...,17.990625,26.915625,26.956250,4.0,31.0,0.885714,Wyoming,10.0,23.0,0.696970
1239,3/20/15,Wyoming,Northern Iowa,2015,N,L,54,71,102.778125,97.556250,...,19.327273,25.645455,26.939394,10.0,23.0,0.696970,Northern Iowa,4.0,31.0,0.885714
1240,3/26/15,Arizona,Xavier,2015,N,W,68,60,115.205882,86.835294,...,19.258824,26.823529,35.208824,4.0,34.0,0.894737,Xavier,14.0,23.0,0.621622


In [18]:
conference_records = conference_records[['conference', 'season', 'win_loss_ratio']]
conference_records.rename(columns={'win_loss_ratio': 'conference_strength'}, inplace=True)

C:\Users\amarl\AppData\Local\Temp\ipykernel_18848\1660582053.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conference_records.rename(columns={'win_loss_ratio': 'conference_strength'}, inplace=True)


In [22]:
tourney_data.columns

Index(['date', 'team', 'opponent', 'season', 'location', 'Win_Loss', 'score',
       'opponent_score', 'adj_offense_team', 'adj_defense_team',
       'offensive_efficiency_team', 'effective_fg_percent_team',
       'turnover_percent_team', 'offensive_rebound_percent_team',
       'free_throw_rate_team', 'opponent_offensive_efficiency_team',
       'opponent_effective_fg_percent_team', 'opponent_turnover_percent_team',
       'opponent_offensive_rebound_percent_team',
       'opponent_free_throw_rate_team', 'adj_offense_opponent',
       'adj_defense_opponent', 'offensive_efficiency_opponent',
       'effective_fg_percent_opponent', 'turnover_percent_opponent',
       'offensive_rebound_percent_opponent', 'free_throw_rate_opponent',
       'opponent_offensive_efficiency_opponent',
       'opponent_effective_fg_percent_opponent',
       'opponent_turnover_percent_opponent',
       'opponent_offensive_rebound_percent_opponent',
       'opponent_free_throw_rate_opponent', 'losses', 'wins',

In [24]:
conference_affiliations = all_games[['team', 'conference', 'season']].drop_duplicates()

In [26]:
conference_affiliations.shape[0]

3540

In [28]:
conference_affiliations.drop_duplicates(subset=['team', 'season']).shape[0] # nice, there are no teams that switch conferences in the same season

3540

In [29]:
tourney_data = pd.merge(tourney_data, conference_affiliations, left_on=['team', 'season'], right_on=['team', 'season'], how='left')

In [33]:
tourney_data = pd.merge(tourney_data, conference_affiliations, left_on=['opponent', 'season'], right_on=['team', 'season'], how='left', suffixes=('', '_opponent'))

In [30]:
tourney_data = pd.merge(tourney_data, conference_records, left_on=['conference', 'season'], right_on=['conference', 'season'], how='left')

In [34]:
tourney_data = pd.merge(tourney_data, conference_records, left_on=['conference_opponent', 'season'], right_on=['conference', 'season'], how='left', suffixes=('', '_opponent'))

In [39]:
tourney_data.drop(columns=['conference_opponent', 'team_opponent'], inplace=True)

In [41]:
tourney_data

,date,team,opponent,season,location,Win_Loss,score,opponent_score,adj_offense_team,adj_defense_team,...,opponent_free_throw_rate_opponent,losses,wins,win_loss_ratio,losses_opponent,wins_opponent,win_loss_ratio_opponent,conference,conference_strength,conference_strength_opponent
0,3/17/23,Arizona St.,TCU,2023,N,L,70,72,105.770588,93.755882,...,30.539394,13.0,23.0,0.638889,13.0,22.0,0.628571,P12,0.618056,0.797297
1,3/17/23,TCU,Arizona St.,2023,N,W,72,70,110.433333,92.660606,...,34.017647,13.0,22.0,0.628571,13.0,23.0,0.638889,B12,0.797297,0.618056
2,3/23/23,Arkansas,Connecticut,2023,N,L,65,88,110.545455,91.821212,...,38.945455,14.0,22.0,0.611111,8.0,31.0,0.794872,SEC,0.709360,0.681159
3,3/23/23,Connecticut,Arkansas,2023,N,W,88,65,118.684848,92.387879,...,39.651515,8.0,31.0,0.794872,14.0,22.0,0.611111,BE,0.681159,0.709360
4,3/18/23,Arkansas,Kansas,2023,N,W,72,71,110.545455,91.821212,...,31.544118,14.0,22.0,0.611111,8.0,28.0,0.777778,SEC,0.709360,0.797297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237,3/19/15,Wofford,Arkansas,2015,N,L,53,56,102.521875,97.062500,...,39.347059,7.0,26.0,0.787879,9.0,27.0,0.750000,SC,0.356436,0.689474
1238,3/20/15,Northern Iowa,Wyoming,2015,N,W,71,54,114.212121,92.445455,...,26.956250,4.0,31.0,0.885714,10.0,23.0,0.696970,MVC,0.601562,0.563492
1239,3/20/15,Wyoming,Northern Iowa,2015,N,L,54,71,102.778125,97.556250,...,26.939394,10.0,23.0,0.696970,4.0,31.0,0.885714,MWC,0.563492,0.601562
1240,3/26/15,Arizona,Xavier,2015,N,W,68,60,115.205882,86.835294,...,35.208824,4.0,34.0,0.894737,14.0,23.0,0.621622,P12,0.718563,0.731343


In [40]:
tourney_data.to_sql('tourney_games_with_season_conference_records', sqlalchemy.create_engine(os.getenv("LINODE_CONN_STRING")), if_exists='replace', index=False, schema='sports_data')

445